<a href="https://colab.research.google.com/github/Tortyfishguy/triage-bot/blob/main/Train_triage_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import files


uploaded = files.upload()

Saving esi_dataset.csv to esi_dataset (2).csv


In [6]:
import pandas as pd

# โหลด dataset
df = pd.read_csv("esi_dataset.csv")

# แปลงระดับ ESI ให้เริ่มที่ 0 (0-4 แทน 1-5)
df["esi_level"] = df["esi_level"] - 1

# แสดงตัวอย่างข้อมูล
df.head()


,text,esi_level
0,หมดสติ,0
1,เจ็บหน้าอกเฉียบพลัน,0
2,หายใจไม่ออก,0
3,แขนขาอ่อนแรงทันที,1
4,เวียนหัวบ้านหมุน,2


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "airesearch/wangchanberta-base-att-spm-uncased"

# โหลด tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# โหลดโมเดลพร้อมกำหนด output เป็น 5 class
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=5)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def tokenize_data(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

from datasets import Dataset

# แปลง Pandas DataFrame เป็น HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Tokenize dataset
dataset = dataset.map(tokenize_data, batched=True)

# แบ่ง Train-Test
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]


Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./esi_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-bdf919cca0d0>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
import os
os.environ["WANDB_DISABLED"] = "true"



In [14]:
from transformers import AutoTokenizer

# โหลด Tokenizer
tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")

# ข้อความตัวอย่าง
texts = ["ปวดหัวมา 3 วัน", "แน่นหน้าอก", "เวียนหัวตอนเช้า"]

# ทำ Tokenization
tokenized_inputs = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt")

# สร้าง dataset
train_dataset = {
    "input_ids": tokenized_inputs["input_ids"],
    "attention_mask": tokenized_inputs["attention_mask"],
    "labels": [0, 1, 2]  # ใส่ label ที่ถูกต้อง
}



In [15]:
texts = df["text"].tolist()  # ดึงข้อความจาก DataFrame
tokenized_inputs = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt")


In [17]:
import pandas as pd

# สร้าง DataFrame ตัวอย่าง
df = pd.DataFrame({
    "text": ["ปวดหัว", "แน่นหน้าอก", "อาเจียน"],
    "label": [2, 1, 3]  # ค่า ESI หรือค่าป้ายกำกับที่ถูกต้อง
})

# กำหนด labels
labels = df["label"].tolist()
print(labels)  # ตรวจสอบว่ามีค่าจริงไหม


[2, 1, 3]


In [18]:
from datasets import Dataset
from transformers import AutoTokenizer

# โหลด Tokenizer
tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")

# Tokenize ข้อความ
tokenized_inputs = tokenizer(df["text"].tolist(), padding=True, truncation=True, max_length=256)

# สร้าง Dataset
train_dataset = Dataset.from_dict({
    "input_ids": tokenized_inputs["input_ids"],
    "attention_mask": tokenized_inputs["attention_mask"],
    "labels": labels  # ใส่ labels ที่กำหนดไว้ก่อนหน้า
})

print(train_dataset)


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3
})


In [20]:
train_dataset = Dataset.from_dict({
    "input_ids": tokenized_inputs["input_ids"],
    "attention_mask": tokenized_inputs["attention_mask"],
    "labels": labels  # <- เพิ่ม labels ตรงนี้
})


In [21]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased", num_labels=5)


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# ข้อมูลตัวอย่าง
texts = ["ปวดหัว", "แน่นหน้าอก", "อาเจียน", "เวียนหัว", "หายใจไม่ออก"]
labels = [2, 1, 3, 2, 1]  # ค่า ESI (สมมติ)

# แบ่งข้อมูล train/test
train_texts, eval_texts, train_labels, eval_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# โหลด Tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")

# Tokenize ข้อมูล
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
eval_encodings = tokenizer(eval_texts, truncation=True, padding=True, max_length=256)

# สร้าง Dataset สำหรับ train และ eval
train_dataset = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "labels": train_labels
})

eval_dataset = Dataset.from_dict({  # <-- สร้าง eval_dataset ตรงนี้
    "input_ids": eval_encodings["input_ids"],
    "attention_mask": eval_encodings["attention_mask"],
    "labels": eval_labels
})

print(train_dataset)
print(eval_dataset)



Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 4
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1
})


In [25]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"  # ✅ ปิดการใช้งาน wandb
)


In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # ✅ ใส่ eval_dataset ที่สร้างไว้
    compute_metrics=compute_metrics,
)


In [27]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = (predictions == torch.tensor(labels)).float().mean().item()
    return {"accuracy": accuracy}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,  # ✅ ใช้ฟังก์ชันนี้ในการประเมิน
)


In [28]:
trainer.train()


Step,Training Loss


TrainOutput(global_step=3, training_loss=1.6574594179789226, metrics={'train_runtime': 18.6935, 'train_samples_per_second': 0.642, 'train_steps_per_second': 0.16, 'total_flos': 30834157320.0, 'train_loss': 1.6574594179789226, 'epoch': 3.0})

In [40]:
import torch  # ✅ เพิ่มบรรทัดนี้
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ฟังก์ชันคาดการณ์ ESI
def predict_esi(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    outputs = model(**inputs)
    predicted_esi = torch.argmax(outputs.logits, dim=1).item() + 1  # ✅ torch ถูก import แล้ว
    return predicted_esi

# ทดสอบ
test_texts = ["ปวดหัว 1 ชั่วโมง", "หมดสติ", "เวียนหัวบ้านหมุน"]
for text in test_texts:
    print(f"อาการ: {text} -> ESI: {predict_esi(text)}")


อาการ: ปวดหัว 1 ชั่วโมง -> ESI: 3
อาการ: หมดสติ -> ESI: 2
อาการ: เวียนหัวบ้านหมุน -> ESI: 3


In [39]:
# ลองให้โมเดลทำนาย ESI สำหรับข้อความใน Training Set
test_texts = ["หมดสติ", "หมดสติ 1 นาที", "หมดสติไปแล้วฟื้น", "ไม่มีสติ", "หายใจไม่ออก"]
for text in test_texts:
    print(f"อาการ: {text} -> ESI: {predict_esi(text)}")


อาการ: หมดสติ -> ESI: 2
อาการ: หมดสติ 1 นาที -> ESI: 3
อาการ: หมดสติไปแล้วฟื้น -> ESI: 2
อาการ: ไม่มีสติ -> ESI: 3
อาการ: หายใจไม่ออก -> ESI: 3


In [42]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./esi_model",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,  # 🔥 เพิ่ม epochs ให้ model เรียนรู้มากขึ้น
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()  # 🚀 เริ่ม Train ใหม่



Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.075316,0.000000
2,No log,1.975935,0.000000
3,No log,2.195944,0.000000
4,No log,2.330598,0.000000
5,No log,2.321998,0.000000


TrainOutput(global_step=5, training_loss=0.8952128410339355, metrics={'train_runtime': 182.1388, 'train_samples_per_second': 0.11, 'train_steps_per_second': 0.027, 'total_flos': 51390262200.0, 'train_loss': 0.8952128410339355, 'epoch': 5.0})

In [43]:
model.save_pretrained("./esi_model")
tokenizer.save_pretrained("./esi_model")


('./esi_model/tokenizer_config.json',
 './esi_model/special_tokens_map.json',
 './esi_model/sentencepiece.bpe.model',
 './esi_model/added_tokens.json',
 './esi_model/tokenizer.json')

In [47]:
from transformers import AutoModelForSequenceClassification

# โหลดโมเดลจากไฟล์ที่บันทึกไว้
model = AutoModelForSequenceClassification.from_pretrained("./esi_model")

# บันทึกโมเดลใหม่เป็น pytorch_model.bin
model.save_pretrained("./esi_model", safe_serialization=False)



In [57]:
import os
os.listdir("./esi_model")


['tokenizer_config.json',
 'sentencepiece.bpe.model',
 'checkpoint-1',
 'config.json',
 'pytorch_model.bin',
 'checkpoint-5',
 'model.safetensors',
 'tokenizer.json',
 'checkpoint-2',
 'special_tokens_map.json',
 'checkpoint-3',
 'checkpoint-4',
 'runs']

In [3]:
!pip install transformers datasets torch scikit-learn pandas numpy tqdm

In [49]:
import shutil
shutil.make_archive("esi_model", 'zip', "./esi_model")


'/content/esi_model.zip'

In [50]:
from google.colab import files
files.download("esi_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [62]:
import torch
from transformers import AutoModelForSequenceClassification

# โหลดโมเดล FP32 ที่ถูกต้อง
model = AutoModelForSequenceClassification.from_pretrained("esi_model")

# แปลงเป็น Half-Precision (FP16)
model.half()

# บันทึกเป็นไฟล์ใหม่
model.save_pretrained("esi_model_fp16")

print("✅ แปลงโมเดลเป็น FP16 และบันทึกไฟล์ใหม่สำเร็จ!")


✅ แปลงโมเดลเป็น FP16 และบันทึกไฟล์ใหม่สำเร็จ!


In [63]:
import os
print(os.listdir("esi_model"))


['tokenizer_config.json', 'sentencepiece.bpe.model', 'checkpoint-1', 'config.json', 'pytorch_model.bin', 'checkpoint-5', 'model.safetensors', 'tokenizer.json', 'checkpoint-2', 'special_tokens_map.json', 'checkpoint-3', 'checkpoint-4', 'runs']


In [66]:
import torch

# โหลดโมเดลที่มีอยู่
model_path = "esi_model/pytorch_model.bin"
model = torch.load(model_path, map_location="cpu")

# แปลงโมเดลเป็น Half-Precision (FP16)
model = {k: v.half() for k, v in model.items()}

# บันทึกโมเดลที่ลดขนาดแล้ว
new_model_path = "esi_model/pytorch_model_fp16.bin"
torch.save(model, new_model_path)

print(f"✅ โมเดลถูกบันทึกใหม่เป็น FP16: {new_model_path}")


✅ โมเดลถูกบันทึกใหม่เป็น FP16: esi_model/pytorch_model_fp16.bin


In [67]:
model = torch.load("esi_model/pytorch_model_fp16.bin", map_location="cpu")


In [71]:
import torch
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "./esi_model", torch_dtype=torch.float16
)
print("🚀 โหลดโมเดล FP16 สำเร็จ!")



🚀 โหลดโมเดล FP16 สำเร็จ!


In [72]:
rm esi_model/pytorch_model.bin


In [74]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_PATH = "./esi_model"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=5,
    torch_dtype=torch.float16  # ✅ บอกให้ใช้ FP16
)

print("✅ โหลดโมเดล FP16 สำเร็จ!")



✅ โหลดโมเดล FP16 สำเร็จ!


In [76]:
!zip -r esi_model.zip esi_model/


	zip warning: Local Version Needed To Extract does not match CD: checkpoint-5/rng_state.pth
	zip warning: Local Version Needed To Extract does not match CD: checkpoint-5/optimizer.pt
	zip warning: Local Version Needed To Extract does not match CD: checkpoint-2/scheduler.pt
	zip warning: Local Version Needed To Extract does not match CD: checkpoint-2/config.json
	zip warning: Local Version Needed To Extract does not match CD: checkpoint-2/trainer_state.json
	zip warning: Local Version Needed To Extract does not match CD: checkpoint-2/training_args.bin
	zip warning: Local Version Needed To Extract does not match CD: checkpoint-2/model.safetensors
	zip warning: Local Version Needed To Extract does not match CD: checkpoint-2/rng_state.pth
	zip warning: Local Version Needed To Extract does not match CD: checkpoint-2/optimizer.pt
	zip warning: Local Version Needed To Extract does not match CD: checkpoint-3/scheduler.pt
	zip warning: Local Version Needed To Extract does not match CD: checkpoi

In [79]:
!unzip -l esi_model.zip



Archive:  esi_model.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
        0  2025-03-17 11:32   checkpoint-1/
        0  2025-03-17 11:33   checkpoint-2/
        0  2025-03-17 11:33   checkpoint-3/
        0  2025-03-17 11:48   checkpoint-4/
        0  2025-03-17 11:49   checkpoint-5/
        0  2025-03-17 11:10   runs/
     1825  2025-03-17 12:01   tokenizer_config.json
   904693  2025-03-17 12:01   sentencepiece.bpe.model
     1031  2025-03-17 12:07   config.json
421059630  2025-03-17 12:07   pytorch_model.bin
421017156  2025-03-17 12:01   model.safetensors
  2179643  2025-03-17 12:01   tokenizer.json
      365  2025-03-17 12:01   special_tokens_map.json
     1064  2025-03-17 11:53   checkpoint-1/scheduler.pt
     1065  2025-03-17 11:52   checkpoint-1/config.json
      961  2025-03-17 11:53   checkpoint-1/trainer_state.json
     5240  2025-03-17 11:52   checkpoint-1/training_args.bin
421017156  2025-03-17 11:52   checkpoint-1/model.safetensors
    13990  2

In [81]:
!rm -rf esi_model/checkpoint-*   # ลบโฟลเดอร์ checkpoint ทั้งหมด
!rm esi_model/model.safetensors  # ลบ model.safetensors
!rm -rf esi_model/runs           # ลบโฟลเดอร์ runs


In [82]:
!zip -r esi_model_clean.zip esi_model/


  adding: esi_model/ (stored 0%)
  adding: esi_model/tokenizer_config.json (deflated 80%)
  adding: esi_model/sentencepiece.bpe.model (deflated 57%)
  adding: esi_model/config.json (deflated 54%)
  adding: esi_model/pytorch_model_fp16.bin (deflated 8%)
  adding: esi_model/tokenizer.json (deflated 74%)
  adding: esi_model/special_tokens_map.json (deflated 51%)


In [83]:
from google.colab import files
files.download("esi_model_clean.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>